In [1]:
import numpy as np
import time
from matplotlib import pyplot as plt

In [106]:
def strassen_alg(A1, B1):
    global nops2
    nops2 = 0
    def strassen_core(A, B):
        global nops2

        if(A.shape[0] == 1):

            C = np.multiply(A, B.transpose())
            #print(C.shape)
            return C
        
        n_prev = A.shape[0]
        m_prev = B.shape[1]

        """
        if(A.shape[0] % 2):
            A = np.pad(A, ((0, 1), (0, 0)), mode = "constant", constant_values = 0)
        if(A.shape[1] % 2):
            A = np.pad(A, ((0, 0), (0, 1)), mode = "constant", constant_values = 0)
        if(B.shape[0] % 2):
            B = np.pad(B, ((0, 1), (0, 0)), mode = "constant", constant_values = 0)
        if(B.shape[1] % 2):
            B = np.pad(B, ((0, 0), (0, 1)), mode = "constant", constant_values = 0)

            
        w1 = A.shape[0]//2
        k1 = A.shape[1]//2
        w2 = B.shape[0]//2
        k2 = B.shape[1]//2
        """
        
        mid = A.shape[0]//2

        A00 = A[:mid, :mid]
        A01 = A[:mid, mid:]
        A10 = A[mid:, :mid]
        A11 = A[mid:, mid:]

        B00 = B[:mid, :mid]
        B01 = B[:mid, mid:]
        B10 = B[mid:, :mid]
        B11 = B[mid:, mid:]

        M1 = strassen_core(np.add(A00, A11), np.add(B00, B11))
        M2 = strassen_core(np.add(A10, A11), B00)
        M3 = strassen_core(A00, np.subtract(B01, B11))
        M4 = strassen_core(A11, np.subtract(B10, B00))
        M5 = strassen_core(np.add(A00, A01), B11)
        M6 = strassen_core(np.subtract(A10, A00), np.add(B00, B01))
        M7 = strassen_core(np.subtract(A01, A11), np.add(B10, B11))

        result = np.zeros((2 * mid, 2 * mid), dtype=np.longdouble)
        result[:mid, :mid] = np.add(np.add(M1, M4), np.subtract(M7, M5))
        result[:mid, mid:] = np.add(M3, M5)
        result[mid:, :mid] = np.add(M2, M4)
        result[mid:, mid:] = np.add(np.subtract(M1, M2), np.add(M3, M6))
        nops2 += mid*mid * 18
        return result[:n_prev, :m_prev]
    
    pot = 1
    max_shape = max(max(A1.shape), max(B1.shape))
    while(pot < max_shape):
        pot *= 2
    B1_prev = B1.shape[1]
    A1_prev = A1.shape[0]
    B1 = np.pad(B1, ((0, pot - B1.shape[0]), (0, pot - B1.shape[1])), mode = "constant", constant_values = 0)
    A1 = np.pad(A1, ((0, pot - A1.shape[0]), (0, pot - A1.shape[1])), mode = "constant", constant_values = 0)
    res1 = strassen_core(A1, B1)
    return res1[:A1_prev, :B1_prev]

In [ ]:
def inverse_recursive(A1, mult_alg):
    n = A1.shape[0]
    if(n == 1):
        return 1/A1
    mid = n//2
    
    A11 = A1[:mid, :mid]
    A12 = A1[:mid, mid:]
    A21 = A1[mid:, :mid]
    A22 = A1[mid:, mid:]

    A11_inv = inverse_recursive(A11, mult_alg)

    S22 = np.subtract(A22, mult_alg(mult_alg(A21, A11_inv), A12))

    S22_inv = inverse_recursive(S22, mult_alg)
    
    B11 = mult_alg(A11_inv, np.add(np.identity(mid), mult_alg(mult_alg(A12, S22_inv), mult_alg(A21, A11_inv))))
    B12 = (-1) * mult_alg(mult_alg(A11_inv, A12), S22_inv)
    B21 = (-1) * mult_alg(mult_alg(S22_inv, A21), A11_inv)
    B22 = S22_inv
    result = np.zeros((n, n), dtype=np.longdouble)
    result[:mid, :mid] = B11
    result[:mid, mid:] = B12
    result[mid:, :mid] = B21
    result[mid:, mid:] = B22
    return result[:n, :n]



In [125]:
def LU_factor(A, mult_alg):

    n = A.shape[0]
    if(n == 1):
        return A, np.array([[1]])
    
    mid = n//2
    A11 = A[:mid, :mid]
    A12 = A[:mid, mid:]
    A21 = A[mid:, :mid]
    A22 = A[mid:, mid:]
    L11, U11 = LU_factor(A11, mult_alg)
    U11_inv = inverse_recursive(U11, mult_alg)
    L21 = mult_alg(A21, U11_inv)
    L11_inv = inverse_recursive(L11, mult_alg)
    U12 = mult_alg(L11_inv, A12)
    S = np.subtract(A22, mult_alg(mult_alg(mult_alg(A21, U11_inv), L11_inv), A12))
    Ls, Us = LU_factor(S, mult_alg)
    U22 = Us
    L22 = Ls
    U_res = np.zeros((n, n), dtype=np.longdouble)
    U_res[:mid, :mid] = U11
    U_res[:mid, mid:] = U12
    U_res[mid:, mid:] = U22
    L_res = np.zeros((n, n), dtype=np.longdouble)
    L_res[:mid, :mid] = L11
    L_res[mid:, :mid] = L21
    L_res[mid:, mid:] = L22
    return L_res[:n, :n], U_res[:n, :n]


In [132]:
rozm = 5

rng = np.random.default_rng()
x = (rng.random((100, 100)) * (1 - 1e-8) + 1e-8).astype(np.longdouble)
y = (rng.random((100, 100)) * (1 - 1e-8) + 1e-8).astype(np.longdouble)

XX = strassen_alg(x, y)
dupa = XX - np.matmul(x, y)
print(dupa[abs(dupa) > 1e-4])

print("\n")
my_inv = inverse_recursive(x, strassen_alg)
delta = abs(my_inv - np.linalg.inv(x.astype(np.double)))
print(delta[delta >= 1e-7])
L1, U1 = LU_factor(x, strassen_alg)
print(L1)
print("\n")
print(U1)
delta_LU = abs( x - np.matmul(L1.astype(np.double), U1.astype(np.double)))
print("\n")
print(delta_LU.sum())



[]


[]
[[ 0.47671378  0.          0.         ...  0.          0.
   0.        ]
 [ 0.57571144  0.13640441  0.         ...  0.          0.
   0.        ]
 [ 0.1163409   0.76919815 -0.49827594 ...  0.          0.
   0.        ]
 ...
 [ 0.41723431  0.05266672 -0.4280962  ...  2.99973782  0.
   0.        ]
 [ 0.20909085  0.32698463 -0.65914854 ...  3.88284877 -4.60024186
   0.        ]
 [ 0.97000009  0.2592264  -1.16217195 ...  4.99209631 -6.08037317
  -1.18436545]]


[[ 1.          0.03350537  1.29941786 ...  0.5255344   1.58759392
   0.30689257]
 [ 0.          1.          1.32309665 ...  2.79805839 -0.04697182
   5.61046178]
 [ 0.          0.          1.         ...  3.73172339  0.05907908
   7.1090065 ]
 ...
 [ 0.          0.          0.         ...  1.          2.37703846
   2.9184739 ]
 [ 0.          0.          0.         ...  0.          1.
   0.60449847]
 [ 0.          0.          0.         ...  0.          0.
   1.        ]]


5.210915423747316e-08
